In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import re

import torch
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

from prover.lean.verifier import Lean4ServerScheduler



In [5]:
!gpustat

rnn.ist.berkeley.edu  Mon Feb  3 11:32:12 2025  535.104.12
[0] NVIDIA RTX A6000 | 27'C,   0 % |   469 / 49140 MB |
[1] NVIDIA RTX A6000 | 54'C,  98 % |  9878 / 49140 MB | oliveradk(9406M)
[2] NVIDIA RTX A6000 | 31'C,   0 % |   466 / 49140 MB |
[3] NVIDIA RTX A6000 | 30'C,   0 % | 44218 / 49140 MB | lukebailey(43732M)
[4] NVIDIA RTX A6000 | 32'C,   0 % |   466 / 49140 MB |
[5] NVIDIA RTX A6000 | 31'C,   0 % |   466 / 49140 MB |
[6] NVIDIA RTX A6000 | 29'C,   0 % |   466 / 49140 MB |
[7] NVIDIA RTX A6000 | 32'C,   0 % |   567 / 49140 MB | ebronstein(28M)


In [4]:
#model_name = "deepseek-ai/DeepSeek-Prover-V1.5-RL"
model_name = "../../models/deepseek-prover-RL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LLM(model=model_name, max_num_batched_tokens=8192, seed=1, trust_remote_code=True, dtype="auto")

#lean4_scheduler = Lean4ServerScheduler(max_concurrent_requests=1, timeout=300, memory_limit=10, name='verifier')
lean4_scheduler = Lean4ServerScheduler(max_concurrent_requests=1, timeout=300, name='verifier')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 02-03 11:24:46 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='../../models/deepseek-prover-RL', speculative_config=None, tokenizer='../../models/deepseek-prover-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=1)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 02-03 11:24:46 utils.py:608] Found nccl from library /home/lukebailey/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 02-03 11:24:46 selector.py:77] Cannot use FlashAttention backend because the flash_attn package is not found. Please install it for better performance.
INFO 02-03 11:24:46 selector.py:33] Using XFormers backend.
INFO 02-03 11:25:28 model_runner.py:173] Loading model weights took 12.8725 GB
INFO 02-03 11:25:30 gpu_executor.py:119] # GPU blocks: 3858, # CPU blocks: 546
INFO 02-03 11:25:31 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-03 11:25:31 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory

In [4]:
# Setting up the problem


prompt = r'''Complete the following Lean 4 code:

```lean4
'''

code_prefix = r'''import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
'''

In [5]:
sampling_params = SamplingParams(
    temperature=1.0,
    max_tokens=2048,
    top_p=0.95,
    n=1,
)
model_inputs = [prompt + code_prefix]
model_outputs = model.generate(
    model_inputs,
    sampling_params,
    use_tqdm=True,
)
result = prompt + code_prefix + model_outputs[0].outputs[0].text
print(result)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.09s/it]

Complete the following Lean 4 code:

```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.succ_inj', one_mul]
  have h₃ : r ^ 2 = 3 := by nlinarith
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply or_iff_not_imp_right.2
    intro h
    field_simp [h, h₃] at h₁ h₂ ⊢
    nlinarith
  simp_all
```


In [6]:
hack_correct = prompt + code_prefix + '''  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄
```
'''

print(hack_correct)

Complete the following Lean 4 code:

```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄
```



In [8]:
# Check if this is correct
formatted_result = re.search(r'```lean4\n(.*?)\n```', hack_correct, re.DOTALL).group(1)
request_id_list = lean4_scheduler.submit_all_request([formatted_result])
outputs_list = lean4_scheduler.get_all_request_outputs(request_id_list)
print(outputs_list[0])
print(outputs_list[0]['complete'])

{'sorries': [], 'tactics': [], 'errors': [], 'warnings': [{'severity': 'warning', 'pos': {'line': 14, 'column': 7}, 'endPos': {'line': 14, 'column': 10}, 'data': "unused variable `h₁'`\nnote: this linter can be disabled with `set_option linter.unusedVariables false`"}, {'severity': 'warning', 'pos': {'line': 15, 'column': 7}, 'endPos': {'line': 15, 'column': 10}, 'data': "unused variable `h₂'`\nnote: this linter can be disabled with `set_option linter.unusedVariables false`"}, {'severity': 'warning', 'pos': {'line': 19, 'column': 35}, 'endPos': {'line': 19, 'column': 38}, 'data': 'Used `tac1 <;> tac2` where `(tac1; tac2)` would suffice\nnote: this linter can be disabled with `set_option linter.unnecessarySeqFocus false`'}, {'severity': 'warning', 'pos': {'line': 20, 'column': 15}, 'endPos': {'line': 20, 'column': 18}, 'data': 'Used `tac1 <;> tac2` where `(tac1; tac2)` would suffice\nnote: this linter can be disabled with `set_option linter.unnecessarySeqFocus false`'}], 'infos': [], 's

In [11]:
formatted_result = re.search(r'```lean4\n(.*?)\n```', hack_correct, re.DOTALL).group(1)
request_id_list = lean4_scheduler.submit_all_request([formatted_result])

In [12]:
request_id_list 

[3]

In [9]:
# Check if this is correct
formatted_result = re.search(r'```lean4\n(.*?)\n```', result, re.DOTALL).group(1)
request_id_list = lean4_scheduler.submit_all_request([formatted_result])
outputs_list = lean4_scheduler.get_all_request_outputs(request_id_list)
print(outputs_list[0])

{'sorries': [], 'tactics': [], 'errors': [{'severity': 'error', 'pos': {'line': 18, 'column': 4}, 'endPos': {'line': 18, 'column': 13}, 'data': 'linarith failed to find a contradiction\ncase h1.h\na r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₂ : a * r ^ 3 = 6\nh₃ : r ^ 2 = 3\nh : ¬a = -(2 / √3)\nh₁ : a * r = 2\na✝ : a * √3 < 2\n⊢ False\nfailed'}], 'warnings': [], 'infos': [], 'system_messages': '', 'system_errors': None, 'ast': {}, 'verified_code': "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)\n  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by\n  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.succ_inj', one_mul]\n  have h₃ : r 

# Generating multiple proofs

In [10]:
proofs = []
# Generate 10 proofs
from tqdm import tqdm
for i in tqdm(range(10), desc="Generating proofs"):
    sampling_params = SamplingParams(
        temperature=1.0,
        max_tokens=2048,
        top_p=0.95,
        n=1,
    )
    model_inputs = [prompt + code_prefix]
    model_outputs = model.generate(
        model_inputs,
        sampling_params,
        use_tqdm=False,
    )
    result = prompt + code_prefix + model_outputs[0].outputs[0].text
    formatted_result = re.search(r'```lean4\n(.*?)\n```', result, re.DOTALL).group(1)
    proofs.append(formatted_result)

Generating proofs: 100%|██████████| 10/10 [00:34<00:00,  3.45s/it]


In [11]:
# Verify all proofs
correct = []

request_id_list = lean4_scheduler.submit_all_request(proofs)
outputs_list = lean4_scheduler.get_all_request_outputs(request_id_list)

In [16]:
print(outputs_list[0].keys())
print([x["pass"] for x in outputs_list])
print([x["complete"] for x in outputs_list])

dict_keys(['sorries', 'tactics', 'errors', 'warnings', 'infos', 'system_messages', 'system_errors', 'ast', 'verified_code', 'pass', 'complete', 'verify_time'])
[True, True, True, False, True, False, True, False, True, True]
[True, True, True, False, True, False, True, False, True, True]


In [17]:
lean4_scheduler.close()

All 1 LeanServerProcesses stopped


# Investigating returns from verifier

In [11]:
# load up ../results/verified_outputs.pkl
import pickle
with open('./results/verified_outputs_imo.pkl', 'rb') as f:
    verified_outputs = pickle.load(f)


In [18]:

for i in range(len(verified_outputs)):
    question = verified_outputs[i][0]

    for key, value in question.items():
        if key == "errors":
            print(key)
            print(len(value))
            print(value[0])
            print(value[0]["data"])
        else:
            print(key)
            print(value)
            print()

    print("\n==========\n")
    if i > 5:
        break


sorries
[]

tactics
[]

3
{'severity': 'error', 'pos': {'line': 18, 'column': 38}, 'endPos': {'line': 18, 'column': 47}, 'data': 'unsolved goals\ncase hab.hab.hx\na b : ℤ\nh₀ : 0 < a ∧ 0 < b\nh₁ : ¬7 ∣ a\nh₂ : ¬7 ∣ b\nh₃ : ¬7 ∣ a + b\nthis : 0 < b\nh₄ : a + b < 19\nh₅ : a + b < 19\n⊢ 0 ≤ a + b\n\ncase hab.hcd.h\na b : ℤ\nh₀ : 0 < a ∧ 0 < b\nh₁ : ¬7 ∣ a\nh₂ : ¬7 ∣ b\nh₃ : ¬7 ∣ a + b\nthis : 0 < b\nh₄ : a + b < 19\nh₅ : a + b < 19\n⊢ 19 < a\n\ncase hcd.h\na b : ℤ\nh₀ : 0 < a ∧ 0 < b\nh₁ : ¬7 ∣ a\nh₂ : ¬7 ∣ b\nh₃ : ¬7 ∣ a + b\nthis : 0 < b\nh₄ : a + b < 19\nh₅ : a + b < 19\n⊢ 19 < b'}
unsolved goals
case hab.hab.hx
a b : ℤ
h₀ : 0 < a ∧ 0 < b
h₁ : ¬7 ∣ a
h₂ : ¬7 ∣ b
h₃ : ¬7 ∣ a + b
this : 0 < b
h₄ : a + b < 19
h₅ : a + b < 19
⊢ 0 ≤ a + b

case hab.hcd.h
a b : ℤ
h₀ : 0 < a ∧ 0 < b
h₁ : ¬7 ∣ a
h₂ : ¬7 ∣ b
h₃ : ¬7 ∣ a + b
this : 0 < b
h₄ : a + b < 19
h₅ : a + b < 19
⊢ 19 < a

case hcd.h
a b : ℤ
h₀ : 0 < a ∧ 0 < b
h₁ : ¬7 ∣ a
h₂ : ¬7 ∣ b
h₃ : ¬7 ∣ a + b
this : 0 < b
h₄ : a + b < 19
h₅ : a + b 